In [8]:
import pandas as pd
import math
from itertools import combinations

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def compute_mst(points):
    n = len(points)
    edges = []
    for i in range(n):
        for j in range(i + 1, n):
            dist = haversine(points[i][0], points[i][1], points[j][0], points[j][1])
            edges.append((dist, i, j))
    edges.sort()
    parent = list(range(n))
    def find(u):
        while parent[u] != u:
            parent[u] = parent[parent[u]]
            u = parent[u]
        return u
    def union(u, v):
        u_root = find(u)
        v_root = find(v)
        if u_root != v_root:
            parent[v_root] = u_root
    mst_sum = 0
    mst_edges = []
    for edge in edges:
        dist, u, v = edge
        if find(u) != find(v):
            union(u, v)
            mst_sum += dist
            mst_edges.append(edge)
            if len(mst_edges) == n - 1:
                break
    return mst_sum

def convert_time_to_minutes(time_str):
    return int(time_str.split(':')[0]) * 60 + int(time_str.split(':')[1])

def tsp_nearest_neighbor(points):
    n = len(points)
    visited = [False] * n
    path = [0]
    visited[0] = True
    current = 0
    for _ in range(n - 1):
        nearest_dist = float('inf')
        nearest_idx = -1
        for i in range(n):
            if not visited[i]:
                dist = haversine(points[current][0], points[current][1], points[i][0], points[i][1])
                if dist < nearest_dist:
                    nearest_dist = dist
                    nearest_idx = i
        if nearest_idx == -1:
            break
        path.append(nearest_idx)
        visited[nearest_idx] = True
        current = nearest_idx
    path.append(0)
    return path

def check_capacity_constraints(num_shipments, vehicle_capacity):
    # Check if shipments are between 50% and 100% of vehicle capacity
    min_shipments = math.ceil(vehicle_capacity * 0.5)
    max_shipments = vehicle_capacity
    return min_shipments <= num_shipments <= max_shipments

# Load data
store_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Store Location')
store_lat = store_df['Latitute']
store_lon = store_df['Longitude']

shipments_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Shipments_Data')
shipments = []
for _, row in shipments_df.iterrows():
    start, end = row['Delivery Timeslot'].split('-')
    shipments.append({
        'id': row['Shipment ID'],
        'lat': row['Latitude'],
        'lon': row['Longitude'],
        'start': convert_time_to_minutes(start.strip()),
        'end': convert_time_to_minutes(end.strip())
    })

# Priority vehicles (3W, 4W-EV) and regular 4W
vehicles = [
    {'type': '3W', 'remaining': 50, 'capacity': 5, 'max_radius': 15},
    {'type': '4W-EV', 'remaining': 25, 'capacity': 8, 'max_radius': 20},
    {'type': '4W', 'remaining': float('inf'), 'capacity': 25, 'max_radius': float('inf')}
]

# Sort shipments by start time and group them by timeslot
shipments.sort(key=lambda x: x['start'])
timeslot_groups = {}
for shipment in shipments:
    key = (shipment['start'], shipment['end'])
    if key not in timeslot_groups:
        timeslot_groups[key] = []
    timeslot_groups[key].append(shipment)

trips = []

# Process each timeslot group
for (start_time, end_time), group_shipments in timeslot_groups.items():
    current_batch = []
    
    # Process shipments in the current timeslot
    for shipment in group_shipments:
        current_batch.append(shipment)
        
        # Try to create a trip when we have enough shipments
        for vehicle in vehicles:
            if vehicle['remaining'] <= 0:
                continue
                
            if check_capacity_constraints(len(current_batch), vehicle['capacity']):
                points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in current_batch]
                mst_dist = compute_mst(points)
                
                if mst_dist <= vehicle['max_radius'] or vehicle['type'] == '4W':
                    trip_time = (mst_dist * 5) + (len(current_batch) * 10)
                    available_time = end_time - start_time
                    
                    if trip_time <= available_time:
                        trips.append({
                            'shipments': current_batch.copy(),
                            'start': start_time,
                            'end': end_time,
                            'mst_dist': mst_dist,
                            'vehicle': vehicle['type'],
                            'vehicle_capacity': vehicle['capacity'],
                            'vehicle_max_radius': vehicle['max_radius']
                        })
                        vehicle['remaining'] -= 1
                        current_batch = []
                        break
    
    # Handle remaining shipments in the batch
    if current_batch:
        # Try to find the best fitting vehicle for remaining shipments
        for vehicle in vehicles:
            if vehicle['remaining'] <= 0:
                continue
            
            if len(current_batch) <= vehicle['capacity']:
                points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in current_batch]
                mst_dist = compute_mst(points)
                
                if mst_dist <= vehicle['max_radius'] or vehicle['type'] == '4W':
                    trips.append({
                        'shipments': current_batch.copy(),
                        'start': start_time,
                        'end': end_time,
                        'mst_dist': mst_dist,
                        'vehicle': vehicle['type'],
                        'vehicle_capacity': vehicle['capacity'],
                        'vehicle_max_radius': vehicle['max_radius']
                    })
                    vehicle['remaining'] -= 1
                    current_batch = []
                    break

# Sequence shipments in each trip using TSP
for trip in trips:
    points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in trip['shipments']]
    path = tsp_nearest_neighbor(points)
    shipment_indices = path[1:-1]  # exclude store at start and end
    ordered_shipments = [trip['shipments'][i-1] for i in shipment_indices]
    trip['shipments'] = ordered_shipments

# Generate output
output_data = []
for idx, trip in enumerate(trips):
    trip_time = (trip['mst_dist'] * 5) + (len(trip['shipments']) * 10)
    available_time = trip['end'] - trip['start']
    time_uti = trip_time / available_time if available_time != 0 else 0
    capacity_uti = len(trip['shipments']) / trip['vehicle_capacity']
    cov_uti = trip['mst_dist'] / trip['vehicle_max_radius'] if trip['vehicle_max_radius'] != float('inf') else 'N/A'
    
    # Create individual rows for each shipment in the trip
    for shipment in trip['shipments']:
        output_data.append({
            'TRIP_ID': f'T{(idx+1):03}_1',
            'Shipment_ID': shipment['id'],
            'Latitude': shipment['lat'],
            'Longitude': shipment['lon'],
            'TIME_SLOT': f"{int(shipment['start']/60):02d}:00-{int(shipment['end']/60):02d}:00",
            'Shipments': len(trip['shipments']),
            'MST_DIST': round(trip['mst_dist'], 2),
            'TRIP_TIME': round(trip_time, 2),
            'Vehicle_Type': trip['vehicle'],
            'CAPACITY_UTI': round(capacity_uti, 2),
            'TIME_UTI': round(time_uti, 2),
            'COV_UTI': round(cov_uti, 2) if isinstance(cov_uti, (int, float)) else cov_uti
        })

# Convert to DataFrame with specific column order
columns = [
    'TRIP_ID', 'Shipment_ID', 'Latitude', 'Longitude', 'TIME_SLOT',
    'Shipments', 'MST_DIST', 'TRIP_TIME', 'Vehicle_Type', 'CAPACITY_UTI',
    'TIME_UTI', 'COV_UTI'
]

output_df = pd.DataFrame(output_data)
output_df = output_df[columns]  # Reorder columns

# Print formatted output
print("\nFormatted Output:")
print(output_df.to_string(index=False))

# Save to CSV
output_df.to_csv('smartroute_output.csv', index=False)
print("\nOutput has been saved to 'smartroute_output.csv'")


Formatted Output:
TRIP_ID  Shipment_ID  Latitude  Longitude   TIME_SLOT  Shipments  MST_DIST  TRIP_TIME Vehicle_Type  CAPACITY_UTI  TIME_UTI  COV_UTI
 T001_1            3 19.058145  72.834377 07:00-09:00          3     12.91      94.57           3W           0.6      0.63     0.86
 T001_1            8 19.032864  72.838498 07:00-09:00          3     12.91      94.57           3W           0.6      0.63     0.86
 T001_1            7 19.119317  72.862686 07:00-09:00          3     12.91      94.57           3W           0.6      0.63     0.86
 T002_1           19 19.105524  72.897726 07:00-09:00          4     17.41     127.07        4W-EV           0.5      0.85     0.87
 T002_1           24 19.133430  72.913268 07:00-09:00          4     17.41     127.07        4W-EV           0.5      0.85     0.87
 T002_1           25 19.135509  72.908397 07:00-09:00          4     17.41     127.07        4W-EV           0.5      0.85     0.87
 T002_1           23 19.007079  72.825190 07:00-09:00    

/tmp/ipykernel_208396/2140362862.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dLat = math.radians(lat2 - lat1)
/tmp/ipykernel_208396/2140362862.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dLon = math.radians(lon2 - lon1)
/tmp/ipykernel_208396/2140362862.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2


In [8]:
import pandas as pd
import math
from itertools import combinations

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def compute_mst(points):
    n = len(points)
    edges = []
    for i in range(n):
        for j in range(i + 1, n):
            dist = haversine(points[i][0], points[i][1], points[j][0], points[j][1])
            edges.append((dist, i, j))
    edges.sort()
    parent = list(range(n))
    def find(u):
        while parent[u] != u:
            parent[u] = parent[parent[u]]
            u = parent[u]
        return u
    def union(u, v):
        u_root = find(u)
        v_root = find(v)
        if u_root != v_root:
            parent[v_root] = u_root
    mst_sum = 0
    mst_edges = []
    for edge in edges:
        dist, u, v = edge
        if find(u) != find(v):
            union(u, v)
            mst_sum += dist
            mst_edges.append(edge)
            if len(mst_edges) == n - 1:
                break
    return mst_sum

def convert_time_to_minutes(time_str):
    return int(time_str.split(':')[0]) * 60 + int(time_str.split(':')[1])

def tsp_nearest_neighbor(points):
    n = len(points)
    visited = [False] * n
    path = [0]
    visited[0] = True
    current = 0
    for _ in range(n - 1):
        nearest_dist = float('inf')
        nearest_idx = -1
        for i in range(n):
            if not visited[i]:
                dist = haversine(points[current][0], points[current][1], points[i][0], points[i][1])
                if dist < nearest_dist:
                    nearest_dist = dist
                    nearest_idx = i
        if nearest_idx == -1:
            break
        path.append(nearest_idx)
        visited[nearest_idx] = True
        current = nearest_idx
    path.append(0)
    return path

def check_capacity_constraints(num_shipments, vehicle_capacity):
    # Check if shipments are between 50% and 100% of vehicle capacity
    min_shipments = math.ceil(vehicle_capacity * 0.5)
    max_shipments = vehicle_capacity
    return min_shipments <= num_shipments <= max_shipments

# Load data
store_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Store Location')
store_lat = store_df['Latitute']
store_lon = store_df['Longitude']

shipments_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Shipments_Data')
shipments = []
for _, row in shipments_df.iterrows():
    start, end = row['Delivery Timeslot'].split('-')
    shipments.append({
        'id': row['Shipment ID'],
        'lat': row['Latitude'],
        'lon': row['Longitude'],
        'start': convert_time_to_minutes(start.strip()),
        'end': convert_time_to_minutes(end.strip())
    })

# Priority vehicles (3W, 4W-EV) and regular 4W
vehicles = [
    {'type': '3W', 'remaining': 50, 'capacity': 5, 'max_radius': 15},
    {'type': '4W-EV', 'remaining': 25, 'capacity': 8, 'max_radius': 20},
    {'type': '4W', 'remaining': float('inf'), 'capacity': 25, 'max_radius': float('inf')}
]

# Sort shipments by start time and group them by timeslot
shipments.sort(key=lambda x: x['start'])
timeslot_groups = {}
for shipment in shipments:
    key = (shipment['start'], shipment['end'])
    if key not in timeslot_groups:
        timeslot_groups[key] = []
    timeslot_groups[key].append(shipment)

trips = []

# Process each timeslot group
for (start_time, end_time), group_shipments in timeslot_groups.items():
    current_batch = []
    
    # Process shipments in the current timeslot
    for shipment in group_shipments:
        current_batch.append(shipment)
        
        # Try to create a trip when we have enough shipments
        for vehicle in vehicles:
            if vehicle['remaining'] <= 0:
                continue
                
            if check_capacity_constraints(len(current_batch), vehicle['capacity']):
                points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in current_batch]
                mst_dist = compute_mst(points)
                
                if mst_dist <= vehicle['max_radius'] or vehicle['type'] == '4W':
                    trip_time = (mst_dist * 5) + (len(current_batch) * 10)
                    available_time = end_time - start_time
                    
                    if trip_time <= available_time:
                        trips.append({
                            'shipments': current_batch.copy(),
                            'start': start_time,
                            'end': end_time,
                            'mst_dist': mst_dist,
                            'vehicle': vehicle['type'],
                            'vehicle_capacity': vehicle['capacity'],
                            'vehicle_max_radius': vehicle['max_radius']
                        })
                        vehicle['remaining'] -= 1
                        current_batch = []
                        break
    
    # Handle remaining shipments in the batch
    if current_batch:
        # Try to find the best fitting vehicle for remaining shipments
        for vehicle in vehicles:
            if vehicle['remaining'] <= 0:
                continue
            
            if len(current_batch) <= vehicle['capacity']:
                points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in current_batch]
                mst_dist = compute_mst(points)
                
                if mst_dist <= vehicle['max_radius'] or vehicle['type'] == '4W':
                    trips.append({
                        'shipments': current_batch.copy(),
                        'start': start_time,
                        'end': end_time,
                        'mst_dist': mst_dist,
                        'vehicle': vehicle['type'],
                        'vehicle_capacity': vehicle['capacity'],
                        'vehicle_max_radius': vehicle['max_radius']
                    })
                    vehicle['remaining'] -= 1
                    current_batch = []
                    break

# Sequence shipments in each trip using TSP
for trip in trips:
    points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in trip['shipments']]
    path = tsp_nearest_neighbor(points)
    shipment_indices = path[1:-1]  # exclude store at start and end
    ordered_shipments = [trip['shipments'][i-1] for i in shipment_indices]
    trip['shipments'] = ordered_shipments

# Generate output
output_data = []
for idx, trip in enumerate(trips):
    trip_time = (trip['mst_dist'] * 5) + (len(trip['shipments']) * 10)
    available_time = trip['end'] - trip['start']
    time_uti = trip_time / available_time if available_time != 0 else 0
    capacity_uti = len(trip['shipments']) / trip['vehicle_capacity']
    cov_uti = trip['mst_dist'] / trip['vehicle_max_radius'] if trip['vehicle_max_radius'] != float('inf') else 'N/A'
    
    # Create individual rows for each shipment in the trip
    for shipment in trip['shipments']:
        output_data.append({
            'TRIP_ID': f'T{(idx+1):03}_1',
            'Shipment_ID': shipment['id'],
            'Latitude': shipment['lat'],
            'Longitude': shipment['lon'],
            'TIME_SLOT': f"{int(shipment['start']/60):02d}:00-{int(shipment['end']/60):02d}:00",
            'Shipments': len(trip['shipments']),
            'MST_DIST': round(trip['mst_dist'], 2),
            'TRIP_TIME': round(trip_time, 2),
            'Vehicle_Type': trip['vehicle'],
            'CAPACITY_UTI': round(capacity_uti, 2),
            'TIME_UTI': round(time_uti, 2),
            'COV_UTI': round(cov_uti, 2) if isinstance(cov_uti, (int, float)) else cov_uti
        })

# Convert to DataFrame with specific column order
columns = [
    'TRIP_ID', 'Shipment_ID', 'Latitude', 'Longitude', 'TIME_SLOT',
    'Shipments', 'MST_DIST', 'TRIP_TIME', 'Vehicle_Type', 'CAPACITY_UTI',
    'TIME_UTI', 'COV_UTI'
]

output_df = pd.DataFrame(output_data)
output_df = output_df[columns]  # Reorder columns

# Print formatted output
print("\nFormatted Output:")
print(output_df.to_string(index=False))

# Save to CSV
output_df.to_csv('smartroute_output.csv', index=False)
print("\nOutput has been saved to 'smartroute_output.csv'")

/tmp/ipykernel_136044/2140362862.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dLat = math.radians(lat2 - lat1)
/tmp/ipykernel_136044/2140362862.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dLon = math.radians(lon2 - lon1)
/tmp/ipykernel_136044/2140362862.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2



Formatted Output:
TRIP_ID  Shipment_ID  Latitude  Longitude   TIME_SLOT  Shipments  MST_DIST  TRIP_TIME Vehicle_Type  CAPACITY_UTI  TIME_UTI  COV_UTI
 T001_1            3 19.058145  72.834377 07:00-09:00          3     12.91      94.57           3W           0.6      0.63     0.86
 T001_1            8 19.032864  72.838498 07:00-09:00          3     12.91      94.57           3W           0.6      0.63     0.86
 T001_1            7 19.119317  72.862686 07:00-09:00          3     12.91      94.57           3W           0.6      0.63     0.86
 T002_1           19 19.105524  72.897726 07:00-09:00          4     17.41     127.07        4W-EV           0.5      0.85     0.87
 T002_1           24 19.133430  72.913268 07:00-09:00          4     17.41     127.07        4W-EV           0.5      0.85     0.87
 T002_1           25 19.135509  72.908397 07:00-09:00          4     17.41     127.07        4W-EV           0.5      0.85     0.87
 T002_1           23 19.007079  72.825190 07:00-09:00    

In [14]:
import pandas as pd
import math
from itertools import combinations

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def compute_mst(points):
    n = len(points)
    edges = []
    for i in range(n):
        for j in range(i + 1, n):
            dist = haversine(points[i][0], points[i][1], points[j][0], points[j][1])
            edges.append((dist, i, j))
    edges.sort()
    parent = list(range(n))
    def find(u):
        while parent[u] != u:
            parent[u] = parent[parent[u]]
            u = parent[u]
        return u
    def union(u, v):
        u_root = find(u)
        v_root = find(v)
        if u_root != v_root:
            parent[v_root] = u_root
    mst_sum = 0
    mst_edges = []
    for edge in edges:
        dist, u, v = edge
        if find(u) != find(v):
            union(u, v)
            mst_sum += dist
            mst_edges.append(edge)
            if len(mst_edges) == n - 1:
                break
    return mst_sum

def convert_time_to_minutes(time_str):
    return int(time_str.split(':')[0]) * 60 + int(time_str.split(':')[1])

def tsp_nearest_neighbor(points):
    n = len(points)
    visited = [False] * n
    path = [0]
    visited[0] = True
    current = 0
    for _ in range(n - 1):
        nearest_dist = float('inf')
        nearest_idx = -1
        for i in range(n):
            if not visited[i]:
                dist = haversine(points[current][0], points[current][1], points[i][0], points[i][1])
                if dist < nearest_dist:
                    nearest_dist = dist
                    nearest_idx = i
        if nearest_idx == -1:
            break
        path.append(nearest_idx)
        visited[nearest_idx] = True
        current = nearest_idx
    path.append(0)  # Return to store
    return path

def check_capacity_constraints(num_shipments, vehicle_capacity):
    min_shipments = math.ceil(vehicle_capacity * 0.5)
    max_shipments = vehicle_capacity
    return min_shipments <= num_shipments <= max_shipments

# Load data
store_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Store Location')
store_lat = store_df['Latitute'].iloc[0]
store_lon = store_df['Longitude'].iloc[0]

shipments_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Shipments_Data')
shipments = []
for _, row in shipments_df.iterrows():
    start, end = row['Delivery Timeslot'].split('-')
    shipments.append({
        'id': row['Shipment ID'],
        'lat': row['Latitude'],
        'lon': row['Longitude'],
        'start': convert_time_to_minutes(start.strip()),
        'end': convert_time_to_minutes(end.strip())
    })

vehicles = [
    {'type': '3W', 'remaining': 50, 'capacity': 5, 'max_radius': 15},
    {'type': '4W-EV', 'remaining': 25, 'capacity': 8, 'max_radius': 20},
    {'type': '4W', 'remaining': float('inf'), 'capacity': 25, 'max_radius': float('inf')}
]

shipments.sort(key=lambda x: x['start'])
timeslot_groups = {}
for shipment in shipments:
    key = (shipment['start'], shipment['end'])
    if key not in timeslot_groups:
        timeslot_groups[key] = []
    timeslot_groups[key].append(shipment)

trips = []

for (start_time, end_time), group_shipments in timeslot_groups.items():
    current_batch = []
    
    for shipment in group_shipments:
        current_batch.append(shipment)
        
        for vehicle in vehicles:
            if vehicle['remaining'] <= 0:
                continue
                
            if check_capacity_constraints(len(current_batch), vehicle['capacity']):
                points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in current_batch]
                mst_dist = compute_mst(points)
                
                last_delivery = points[-1]
                return_dist = haversine(last_delivery[0], last_delivery[1], store_lat, store_lon)
                total_dist = mst_dist + return_dist  # Include return trip distance
                
                if total_dist > vehicle['max_radius']:  # If distance exceeds max allowed
                    if len(current_batch) > 1:  # Keep previous batch and create new one
                        trips.append({
                            'shipments': current_batch[:-1],
                            'start': start_time,
                            'end': end_time,
                            'mst_dist': mst_dist - return_dist,
                            'vehicle': vehicle['type'],
                            'vehicle_capacity': vehicle['capacity'],
                            'vehicle_max_radius': vehicle['max_radius']
                        })
                        vehicle['remaining'] -= 1
                        current_batch = [shipment]  # Start new batch with current shipment
                
                if check_capacity_constraints(len(current_batch), vehicle['capacity']):
                    trips.append({
                        'shipments': current_batch.copy(),
                        'start': start_time,
                        'end': end_time,
                        'mst_dist': total_dist,
                        'vehicle': vehicle['type'],
                        'vehicle_capacity': vehicle['capacity'],
                        'vehicle_max_radius': vehicle['max_radius']
                    })
                    vehicle['remaining'] -= 1
                    current_batch = []
                    break
    
    if current_batch:
        for vehicle in vehicles:
            if vehicle['remaining'] <= 0:
                continue
            
            if len(current_batch) <= vehicle['capacity']:
                points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in current_batch]
                mst_dist = compute_mst(points)
                
                last_delivery = points[-1]
                return_dist = haversine(last_delivery[0], last_delivery[1], store_lat, store_lon)
                total_dist = mst_dist + return_dist
                
                if total_dist <= vehicle['max_radius'] or vehicle['type'] == '4W':
                    trips.append({
                        'shipments': current_batch.copy(),
                        'start': start_time,
                        'end': end_time,
                        'mst_dist': total_dist,
                        'vehicle': vehicle['type'],
                        'vehicle_capacity': vehicle['capacity'],
                        'vehicle_max_radius': vehicle['max_radius']
                    })
                    vehicle['remaining'] -= 1
                    current_batch = []
                    break

for trip in trips:
    points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in trip['shipments']]
    path = tsp_nearest_neighbor(points)
    shipment_indices = path[1:-1]  
    ordered_shipments = [trip['shipments'][i-1] for i in shipment_indices]
    trip['shipments'] = ordered_shipments

output_data = []
for idx, trip in enumerate(trips):
    trip_time = (trip['mst_dist'] * 5) + (len(trip['shipments']) * 10)
    available_time = trip['end'] - trip['start']
    
    for shipment in trip['shipments']:
        output_data.append({
            'TRIP_ID': f'T{(idx+1):03}_1',
            'Shipment_ID': shipment['id'],
            'TIME_SLOT': f"{int(shipment['start']/60):02d}:00-{int(shipment['end']/60):02d}:00",
            'MST_DIST': round(trip['mst_dist'], 2),
            'TRIP_TIME': round(trip_time, 2),
            'Vehicle_Type': trip['vehicle']
        })

pd.DataFrame(output_data).to_csv('smartroute_output.csv', index=False)
print("Updated output saved to 'smartroute_output.csv'")


Updated output saved to 'smartroute_output.csv'


In [4]:
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c
    
print(haversine(19.075887, 72.877911, 19.0581451, 72.8343773))
print(haversine(19.0581451, 72.8343773, 19.119317, 72.8626863))
print(haversine(19.119317, 72.8626863, 19.075887, 72.877911))

4.982366646082719
7.4240340954953545
5.087267057676537


In [7]:
print(haversine(19.075887, 72.877911, 19.0070794, 72.8251896))
print(haversine(19.0070794, 72.8251896, 19.1334302, 72.9132679))
print(haversine(19.1334302, 72.9132679, 19.075887, 72.877911))

print('--------------------------------')

print(haversine(19.075887, 72.877911, 19.1334302, 72.9132679))
print(haversine(19.1334302, 72.9132679, 19.0070794, 72.8251896))
print(haversine(19.0070794, 72.8251896, 19.075887, 72.877911))

9.44710313410789
16.82469939437282
7.39878039666553
--------------------------------
7.39878039666553
16.82469939437282
9.44710313410789


In [9]:
import pandas as pd
import math
from itertools import combinations

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def compute_mst(points):
    n = len(points)
    edges = []
    for i in range(n):
        for j in range(i + 1, n):
            dist = haversine(points[i][0], points[i][1], points[j][0], points[j][1])
            edges.append((dist, i, j))
    edges.sort()
    parent = list(range(n))
    
    def find(u):
        while parent[u] != u:
            parent[u] = parent[parent[u]]
            u = parent[u]
        return u
    
    def union(u, v):
        u_root = find(u)
        v_root = find(v)
        if u_root != v_root:
            parent[v_root] = u_root
    
    mst_sum = 0
    for dist, u, v in edges:
        if find(u) != find(v):
            union(u, v)
            mst_sum += dist
    return mst_sum

def convert_time_to_minutes(time_str):
    return int(time_str.split(':')[0]) * 60 + int(time_str.split(':')[1])

def check_capacity_constraints(num_shipments, vehicle_capacity):
    return math.ceil(vehicle_capacity * 0.5) <= num_shipments <= vehicle_capacity

# Load data
store_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Store Location')
store_lat = store_df['Latitute'].iloc[0]
store_lon = store_df['Longitude'].iloc[0]

shipments_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Shipments_Data')
shipments = []
for _, row in shipments_df.iterrows():
    start, end = row['Delivery Timeslot'].split('-')
    shipments.append({
        'id': row['Shipment ID'],
        'lat': row['Latitude'],
        'lon': row['Longitude'],
        'start': convert_time_to_minutes(start.strip()),
        'end': convert_time_to_minutes(end.strip())
    })

vehicles = [
    {'type': '3W', 'remaining': 50, 'capacity': 5, 'max_radius': 15},
    {'type': '4W-EV', 'remaining': 25, 'capacity': 8, 'max_radius': 20},
    {'type': '4W', 'remaining': float('inf'), 'capacity': 25, 'max_radius': float('inf')}
]

shipments.sort(key=lambda x: x['start'])
timeslot_groups = {}
for shipment in shipments:
    key = (shipment['start'], shipment['end'])
    if key not in timeslot_groups:
        timeslot_groups[key] = []
    timeslot_groups[key].append(shipment)

trips = []

for (start_time, end_time), group_shipments in timeslot_groups.items():
    current_batch = []
    
    for shipment in group_shipments:
        current_batch.append(shipment)
        
        for vehicle in vehicles:
            if vehicle['remaining'] <= 0:
                continue
                
            if check_capacity_constraints(len(current_batch), vehicle['capacity']):
                points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in current_batch]
                mst_dist = compute_mst(points)
                
                last_delivery = points[-1]
                return_dist = haversine(last_delivery[0], last_delivery[1], store_lat, store_lon)
                total_dist = mst_dist + return_dist  # Include return trip distance
                
                if total_dist > vehicle['max_radius']:  # Split trip if distance exceeds max radius
                    if len(current_batch) > 1:  
                        trips.append({
                            'shipments': current_batch[:-1],
                            'start': start_time,
                            'end': end_time,
                            'mst_dist': mst_dist - return_dist,
                            'vehicle': vehicle['type'],
                            'vehicle_capacity': vehicle['capacity'],
                            'vehicle_max_radius': vehicle['max_radius']
                        })
                        vehicle['remaining'] -= 1
                        current_batch = [shipment]  

                if check_capacity_constraints(len(current_batch), vehicle['capacity']):
                    trips.append({
                        'shipments': current_batch.copy(),
                        'start': start_time,
                        'end': end_time,
                        'mst_dist': total_dist,
                        'vehicle': vehicle['type'],
                        'vehicle_capacity': vehicle['capacity'],
                        'vehicle_max_radius': vehicle['max_radius']
                    })
                    vehicle['remaining'] -= 1
                    current_batch = []
                    break

# Ensure no vehicle exceeds its constraints
# Adjust trips dynamically if they exceed vehicle constraints
for trip in trips:
    while trip['mst_dist'] > trip['vehicle_max_radius']:  # If trip exceeds max radius
        if len(trip['shipments']) > 1:  # Only split if we have more than one shipment
            removed_shipment = trip['shipments'].pop()  # Remove last shipment
            points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in trip['shipments']]
            mst_dist = compute_mst(points)  # Recalculate MST distance
            
            last_delivery = points[-1]
            return_dist = haversine(last_delivery[0], last_delivery[1], store_lat, store_lon)
            trip['mst_dist'] = mst_dist + return_dist  # Update distance
            
            # Store removed shipment in a new trip
            new_trip = {
                'shipments': [removed_shipment],
                'start': trip['start'],
                'end': trip['end'],
                'mst_dist': haversine(store_lat, store_lon, removed_shipment['lat'], removed_shipment['lon']) * 2,  # Direct trip
                'vehicle': trip['vehicle'],
                'vehicle_capacity': trip['vehicle_capacity'],
                'vehicle_max_radius': trip['vehicle_max_radius']
            }
            trips.append(new_trip)
        else:
            break  # Stop adjusting if only one shipment remains

# Generate Output
output_data = []
for idx, trip in enumerate(trips):
    trip_time = (trip['mst_dist'] * 5) + (len(trip['shipments']) * 10)
    available_time = trip['end'] - trip['start']
    
    for shipment in trip['shipments']:
        output_data.append({
            'TRIP_ID': f'T{(idx+1):03}_1',
            'Shipment_ID': shipment['id'],
            'TIME_SLOT': f"{int(shipment['start']/60):02d}:00-{int(shipment['end']/60):02d}:00",
            'MST_DIST': round(trip['mst_dist'], 2),
            'TRIP_TIME': round(trip_time, 2),
            'Vehicle_Type': trip['vehicle']
        })

pd.DataFrame(output_data).to_csv('smartroute_output.csv', index=False)
print("Updated output saved to 'smartroute_output.csv'")

Updated output saved to 'smartroute_output.csv'
